# Calculating entropy of DNA strand (Solute)

## Loading Data
1. Load your data into a MDAnalysis Universe. 

In [ ]:
import MDAnalysis as mda
# set the working dir to the root of repo inorder to use these path
tprfile = "data/md_A4_dna.tpr"
trrfile = "data/md_A4_dna_xf.trr"
u = mda.Universe(tprfile, trrfile)

2. Trim the data to reduce analysis time

In [ ]:
from CodeEntropy.IO import MDAUniverseHelper as MDAHelper
# since this trajectory only contains the DNA strand
selection_string_pre_process = 'all'
start = 3
end = 40
step = 1

reduced_frame = MDAHelper.new_U_select_frame(u,  start, end, step)

reduced_atom = MDAHelper.new_U_select_atom(reduced_frame, selection_string_pre_process)



3. parse the data into a CodeEntropy data object

In [ ]:
from CodeEntropy.ClassCollection import DataContainer as DC
dataContainer = DC.DataContainer(reduced_atom)

## Performing calculation
The total entropy for a system is taken as the sum of seven terms:
$S_{total} = S^{transvib}_{WM} + S^{rovib}_{WM} + S^{transvib}_{R} + S^{rovib}_{R} + S^{transvib}_{UA} + S^{rovib}_{UA} + S^{topo}_{UA}$

### Set Parameters

In [ ]:
from CodeEntropy.FunctionCollection import EntropyFunctions as EF
#Only the part of interest remained
selection_string = "all"
outfile = None
moutFile = None
nmdFile = None
csv_out = None
tScale = 1.0
fScale = 1.0
temper = 300.0 #K
verbose = 3
thread = 4

### Whole-molecule Level 
$S^{transvib}_{WM} + S^{rovib}_{WM}$

In [ ]:
wm_entropyFF, wm_entropyTT = EF.compute_entropy_whole_molecule_level(
    arg_hostDataContainer = dataContainer,
    arg_outFile = outfile,
    arg_selector = selection_string, 
    arg_moutFile = moutFile,
    arg_nmdFile = nmdFile,
    arg_fScale = fScale,
    arg_tScale = tScale,
    arg_temper = temper,
    arg_verbose = verbose
)
print(f"wm_entropyFF = {wm_entropyFF}")
print(f"wm_entropyTT = {wm_entropyTT}")

### Residue level
$S^{transvib}_{R} + S^{rovib}_{R}$

In [ ]:
res_entropyFF, res_entropyTT = EF.compute_entropy_residue_level(
    arg_hostDataContainer = dataContainer,
    arg_outFile = outfile,
    arg_selector = selection_string, 
    arg_moutFile = moutFile,
    arg_nmdFile = nmdFile,
    arg_fScale = fScale,
    arg_tScale = tScale,
    arg_temper = temper,
    arg_verbose = verbose,
)

print(f"res_entropyFF = {res_entropyFF}")
print(f"res_entropyTT = {res_entropyTT}")

### United Atom Level
$S^{transvib}_{UA} + S^{rovib}_{UA}$

In [ ]:
UA_entropyFF, UA_entropyTT, res_df = EF.compute_entropy_UA_level(
    arg_hostDataContainer = dataContainer,
    arg_outFile = outfile,
    arg_selector = selection_string, 
    arg_moutFile = moutFile,
    arg_nmdFile = nmdFile,
    arg_fScale = fScale,
    arg_tScale = tScale,
    arg_temper = temper,
    arg_verbose = verbose,
    arg_csv_out= csv_out,
)
print(f"UA_entropyFF = {UA_entropyFF}")
print(f"UA_entropyTT = {UA_entropyTT}")
print("Per residue entropy:")
print(res_df)

#### United Atom Level Multi process
Use Multiple thread to speed up operation

In [ ]:
UA_entropyFF, UA_entropyTT, res_df = EF.compute_entropy_UA_level_multiprocess(
    arg_hostDataContainer = dataContainer,
    arg_outFile = outfile,
    arg_selector = selection_string, 
    arg_moutFile = moutFile,
    arg_nmdFile = nmdFile,
    arg_fScale = fScale,
    arg_tScale = tScale,
    arg_temper = temper,
    arg_verbose = verbose,
    arg_csv_out= csv_out,
    arg_thread= thread,
)
print(f"UA_entropyFF = {UA_entropyFF}")
print(f"UA_entropyTT = {UA_entropyTT}")
print("Per residue entropy:")
print(res_df)

### Topographical entropy
$S^{topo}_{UA}$

In [ ]:
result_entropyAEM = EF.compute_topographical_entropy_AEM(
    arg_hostDataContainer = dataContainer, 
    arg_selector = selection_string,
    arg_outFile = outfile, 
    arg_verbose = verbose
)

print(f"result_entropyAEM = {result_entropyAEM}")

### Total Entropy

In [ ]:
total = wm_entropyFF + wm_entropyTT + res_entropyFF + res_entropyTT + UA_entropyFF + UA_entropyTT + result_entropyAEM
print(f"Total Entropy = {total} J/mol/K")